使用GPR算法建立Sepal.Length、Sepal.Width、Petal.Length对Petal.Width回归问题的高斯过程回归模型

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 准备基础数据
iris = pd.read_csv("http://image.cador.cn/data/iris.csv")
x,y = iris.drop(columns=['Species','Petal.Width']),iris['Petal.Width']

# 标准化处理
x = x.apply(lambda v:(v-np.mean(v))/np.std(v))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

# 初始化参数
n = x_train.shape[0]
epsilon = 1e-3
theta1 = 1
theta2 = 1
theta3 = 1
learnRate = 0.005

进行迭代求取最优超参

In [2]:
def delta(bgc,delta,y):
    bgc_inv = np.linalg.inv(bgc)
    a = np.sum(np.diag(np.matmul(bgc_inv,delta)))
    b = np.matmul(np.matmul(y,np.matmul(np.matmul(bgc_inv,delta),bgc_inv)),y)
    return 0.5*(a - b)

def bigc(data,t1,t2,t3):
    rows = data.shape[0]
    tmp = np.zeros((rows,rows))
    for e in range(rows):
        x_tmp = data.iloc[e,:]
        tmp[e,:] = np.exp(2*t2)*np.exp(-np.sum((data - x_tmp)**2,axis=1)/(2*np.exp(2*t1)))
    return tmp + np.identity(rows)*np.exp(2*t3)

for i in range(1000):
    bigC = bigc(x_train, theta1, theta2, theta3)
    # 更新theta1
    delta1 = np.zeros((n,n))
    for j in range(n):
        xi = x_train.iloc[j,:]
        deltaX = (x_train - xi)**2
        rsobj = np.sum(deltaX,axis=1)
        delta1[j,:]=np.exp(2*theta2)*np.exp(2*theta2)*np.exp(-rsobj/(2*np.exp(2*theta1)))*rsobj/(2*np.exp(2*theta1))
    
    delta1 = delta(bigC,delta1,y_train)
    theta1=theta1-learnRate*delta1
    
    # 更新theta2
    delta2 = np.zeros((n,n))
    for j in range(n):
        xi = x_train.iloc[j,:]
        deltaX = (x_train - xi)**2
        delta2[j,:] = 2*np.exp(2*theta2)*np.exp(2*theta2)*np.exp(-np.sum(deltaX,axis=1)/(2*np.exp(2*theta1)))
    
    delta2 = delta(bigC,delta2,y_train)
    theta2=theta2-learnRate*delta2
    
    # 更新theta3
    delta3 = np.identity(n)*np.exp(2*theta3)
    delta3 = delta(bigC,delta3,y_train)
    theta3=theta3-learnRate*delta3
    print(i,"---delta1:",delta1,"delta2:",delta2,"delta3:",delta3)
    
    # 当超参数的变化量绝对值的最大值小于给定精度时，退出循环
    if np.max(np.abs([delta1,delta2,delta3])) < epsilon :
        break

0 ---delta1: -15.43597735905513 delta2: 28.94230890212499 delta3: 47.040787150700105
1 ---delta1: -11.202121918475905 delta2: 20.269730245089814 delta3: 46.90575619288189
2 ---delta1: -9.326096699821788 delta2: 15.674590488714756 delta3: 46.591193943104486
3 ---delta1: -8.217168550831605 delta2: 12.588401264999412 delta3: 46.12465918227182
4 ---delta1: -7.424331779759415 delta2: 10.194781115577216 delta3: 45.476895158353685
5 ---delta1: -6.772508024353025 delta2: 8.164291391288439 delta3: 44.581220614061664
6 ---delta1: -6.184085450864703 delta2: 6.363263446252443 delta3: 43.32860502198186
7 ---delta1: -5.617944097305971 delta2: 4.750734611577924 delta3: 41.56226203925314
8 ---delta1: -5.038472004133996 delta2: 3.322759104646913 delta3: 39.08619560434109
9 ---delta1: -4.402313025138282 delta2: 2.076985098144359 delta3: 35.705227454231334
10 ---delta1: -3.6685662796210785 delta2: 1.0003234571120814 delta3: 31.312385183222588
11 ---delta1: -2.8317359033229046 delta2: 0.08175075587269731 

In [3]:
#求得的三个超参数分别为
theta1,theta2,theta3

(1.4767280756922787, 0.5247171125076914, -1.7670980634788682)

基于这些超参数，进行GPR预测

In [4]:
# 进行预测并计算残差平方和
bigC = bigc(x_train, theta1, theta2, theta3)
alpha = np.matmul(np.linalg.inv(bigC),y_train)
ypred = []
ysigma = []
tn = x_test.shape[0]
for j in range(tn):
    xi = x_test.iloc[j,:]
    deltaX = (x_train - xi)**2
    t0 = np.exp(2*theta2)*np.exp(-np.sum(deltaX,axis=1)/(2*np.exp(2*theta1)))
    ypred.append(np.matmul(t0,alpha))
    ysigma.append(np.sqrt(np.exp(2*theta2) - np.matmul(np.matmul(t0,np.linalg.inv(bigC)),t0)))

# 最终得到的残差平方和为
np.sum((y_test.values - ypred)**2)

2.0819543717906575

In [5]:
pd.DataFrame({'y_test':y_test,'ypred':ypred,'sigma':ysigma}).head()

,y_test,ypred,sigma
14,0.2,0.170740,0.114043
98,1.1,0.820464,0.048525
75,1.4,1.410814,0.047854
16,0.4,0.201179,0.067488
131,2.0,2.145182,0.151244
